In [1]:
!pip install thefuzz

In [2]:
!pip install python-Levenshtein

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from thefuzz import fuzz

In [4]:
df = pd.read_csv(r"C:\Users\amira\Dropbox\_MACHINE LEARNING\DataSets\books.csv",on_bad_lines='skip')

In [5]:
df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [6]:
df.isna().sum()

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
  num_pages           0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64

In [7]:
cols = df.columns.tolist()

In [8]:
cols = list(map(str.strip,cols))

In [9]:
cols

['bookID',
 'title',
 'authors',
 'average_rating',
 'isbn',
 'isbn13',
 'language_code',
 'num_pages',
 'ratings_count',
 'text_reviews_count',
 'publication_date',
 'publisher']

In [10]:
df.columns = cols

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  int64  
 6   language_code       11123 non-null  object 
 7   num_pages           11123 non-null  int64  
 8   ratings_count       11123 non-null  int64  
 9   text_reviews_count  11123 non-null  int64  
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.0+ MB


In [12]:
df.set_index('bookID',inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11123 entries, 1 to 45641
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               11123 non-null  object 
 1   authors             11123 non-null  object 
 2   average_rating      11123 non-null  float64
 3   isbn                11123 non-null  object 
 4   isbn13              11123 non-null  int64  
 5   language_code       11123 non-null  object 
 6   num_pages           11123 non-null  int64  
 7   ratings_count       11123 non-null  int64  
 8   text_reviews_count  11123 non-null  int64  
 9   publication_date    11123 non-null  object 
 10  publisher           11123 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 1.0+ MB


In [14]:
df['language_code'].unique()

array(['eng', 'en-US', 'fre', 'spa', 'en-GB', 'mul', 'grc', 'enm',
       'en-CA', 'ger', 'jpn', 'ara', 'nl', 'zho', 'lat', 'por', 'srp',
       'ita', 'rus', 'msa', 'glg', 'wel', 'swe', 'nor', 'tur', 'gla',
       'ale'], dtype=object)

In [15]:
df['publication_date_new'] = pd.to_datetime(df['publication_date'],infer_datetime_format=True,errors = 'coerce')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11123 entries, 1 to 45641
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   title                 11123 non-null  object        
 1   authors               11123 non-null  object        
 2   average_rating        11123 non-null  float64       
 3   isbn                  11123 non-null  object        
 4   isbn13                11123 non-null  int64         
 5   language_code         11123 non-null  object        
 6   num_pages             11123 non-null  int64         
 7   ratings_count         11123 non-null  int64         
 8   text_reviews_count    11123 non-null  int64         
 9   publication_date      11123 non-null  object        
 10  publisher             11123 non-null  object        
 11  publication_date_new  11121 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 1.1+ M

In [17]:
df[df['publication_date_new'].isna()][['publication_date','publication_date_new']]

,publication_date,publication_date_new
bookID,,
31373,11/31/2000,NaT
45531,6/31/1982,NaT


In [18]:
df.head()

,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,publication_date_new
bookID,,,,,,,,,,,,
1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,2006-09-16
2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,2004-09-01
4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,2003-11-01
5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,2004-05-01
8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,2004-09-13


In [27]:
publishers = sorted(df['publisher'].unique().tolist())
#print(f'publishers: {len(publishers)}')
unique_clean = dict()
iter_ps = iter(publishers)
start = next(iter_ps)
#print(f'Start: {start}')
publishers.remove(start)
#print(f'publishers: {len(publishers)}')
while publishers:
    try:
        unique_clean[start] = []
        #print(unique_clean)
        for next_item in publishers:
            if fuzz.WRatio(start,next_item) >= 79:
                #print(f'--{start}-- is similar to --{next_item}-- by --{fuzz.WRatio(start,next_item)}--')
                if not next_item in unique_clean[start]:
                    unique_clean[start].append(next_item)
                    #print(unique_clean)
                publishers.remove(next_item)
        start = next(iter_ps)
    except StopIteration:
        break

In [21]:
unique_clean

{'10/18': ['Editions 10/18', 'Éditions 10/18'],
 '1st World Library': ['1st World Library'],
 'A Harvest Book/Harcourt  Inc.': ['A Harvest Book/Harcourt  Inc.',
  'Harcourt',
  'Harcourt  Inc.(Harvest Book)'],
 'AA World Services': ['AA World Services'],
 'ADV Manga': ['ADV Manga'],
 'AIT Planet Lar': ['AIT Planet Lar'],
 'AMR/Advanced Management Reports': ['AMR/Advanced Management Reports'],
 'AVA Publishing': ['AVA Publishing'],
 'Abacus Books': ['Abacus', 'Abacus Books (London)'],
 'Abingdon Press': ['Abingdon Press'],
 'Abraham Guillen Press': ['Abraham Guillen Press'],
 'Absolute Press': ['Absolute Press'],
 'Academic Press': ['Academic Press'],
 'Academy Chicago Publishers': ['Academy Chicago Publishers'],
 'Ace Book': ['ACE', 'Ace', 'Ace Books'],
 'Ace/SFBC': ['Ace/SFBC'],
 'Acropolis Books (NY)': ['Acropolis Books (NY)'],
 'Adamant Media Corporation': ['Adamant Media Corporation'],
 'Addison Wesley': ['Addison Wesley', 'Addison-Wesley Professional'],
 'Advantage World Press': [

In [22]:
df['title'].unique()

array(['Harry Potter and the Half-Blood Prince (Harry Potter  #6)',
       'Harry Potter and the Order of the Phoenix (Harry Potter  #5)',
       'Harry Potter and the Chamber of Secrets (Harry Potter  #2)', ...,
       'The Ice-Shirt (Seven Dreams #1)', 'Poor People',
       'Las aventuras de Tom Sawyer'], dtype=object)

In [23]:
df['title'].nunique()

10348

In [29]:
check_list =['Bloomsbury (NYC)','Bloomsbury','Bloomsbury Academic','Bloomsbury Academic','Bloomsbury Childrens Books',\
"Bloomsbury Children's Books","Bloomsbury USA Children's Books",'Bloomsbury Paperbacks','Bloomsbury Paperbacks',\
'Bloomsbury Publishing PLC','Bloomsbury Publishing','Bloomsbury UK','Bloomsbury UK','Bloomsbury USA Childrens',\
'Bloomsbury USA']

In [30]:
check_list_1 = ['W. W. Norton & Co.','W.W. Norton  Company','W. W. Norton and Company',\
 'W.W. Norton & Company (NY/London)','W. W. Norton&Co.',\
 'W.W. Norton & Co. (NY/London)']

In [31]:
def checktextSimilarity(checklist):
    s_word = checklist[0]
    for x_word in checklist[1:]:
        result = fuzz.WRatio(s_word,x_word)
        print(f'{s_word!r} matches {x_word!r} by ({result})')

In [32]:
checktextSimilarity(check_list)

'Bloomsbury (NYC)' matches 'Bloomsbury' by (90)
'Bloomsbury (NYC)' matches 'Bloomsbury Academic' by (79)
'Bloomsbury (NYC)' matches 'Bloomsbury Academic' by (79)
'Bloomsbury (NYC)' matches 'Bloomsbury Childrens Books' by (86)
'Bloomsbury (NYC)' matches "Bloomsbury Children's Books" by (86)
'Bloomsbury (NYC)' matches "Bloomsbury USA Children's Books" by (86)
'Bloomsbury (NYC)' matches 'Bloomsbury Paperbacks' by (79)
'Bloomsbury (NYC)' matches 'Bloomsbury Paperbacks' by (79)
'Bloomsbury (NYC)' matches 'Bloomsbury Publishing PLC' by (86)
'Bloomsbury (NYC)' matches 'Bloomsbury Publishing' by (79)
'Bloomsbury (NYC)' matches 'Bloomsbury UK' by (83)
'Bloomsbury (NYC)' matches 'Bloomsbury UK' by (83)
'Bloomsbury (NYC)' matches 'Bloomsbury USA Childrens' by (86)
'Bloomsbury (NYC)' matches 'Bloomsbury USA' by (79)


In [33]:
checktextSimilarity(check_list_1)

'W. W. Norton & Co.' matches 'W.W. Norton  Company' by (81)
'W. W. Norton & Co.' matches 'W. W. Norton and Company' by (80)
'W. W. Norton & Co.' matches 'W.W. Norton & Company (NY/London)' by (86)
'W. W. Norton & Co.' matches 'W. W. Norton&Co.' by (95)
'W. W. Norton & Co.' matches 'W.W. Norton & Co. (NY/London)' by (86)


In [34]:
check_list_2 = ['William Morrow','Morrow','Perennial / William Morrow / HarperCollins','William Morrow',\
'William Morrow & Company  Inc.','William Morrow / Harper','William Morrow Paperbacks',\
'William Morrow; 1ST edition','William Morrow; 1ST edition']

In [36]:
checktextSimilarity(check_list_2)

'William Morrow' matches 'Morrow' by (90)
'William Morrow' matches 'Perennial / William Morrow / HarperCollins' by (90)
'William Morrow' matches 'William Morrow' by (100)
'William Morrow' matches 'William Morrow & Company  Inc.' by (90)
'William Morrow' matches 'William Morrow / Harper' by (90)
'William Morrow' matches 'William Morrow Paperbacks' by (90)
'William Morrow' matches 'William Morrow; 1ST edition' by (90)
'William Morrow' matches 'William Morrow; 1ST edition' by (90)


In [37]:
check_list_3 = ["St. Martin's Minotaur","St. Martin's Minotaur","St. Martin's Paperbacks","St. Martin's Paperback",\
"St. Martin's True Crime","St. Martin's True Crime",'St. Martins Press-3PL','St. Martins Press']

In [38]:
checktextSimilarity(check_list_3)

"St. Martin's Minotaur" matches "St. Martin's Minotaur" by (100)
"St. Martin's Minotaur" matches "St. Martin's Paperbacks" by (68)
"St. Martin's Minotaur" matches "St. Martin's Paperback" by (70)
"St. Martin's Minotaur" matches "St. Martin's True Crime" by (73)
"St. Martin's Minotaur" matches "St. Martin's True Crime" by (73)
"St. Martin's Minotaur" matches 'St. Martins Press-3PL' by (62)
"St. Martin's Minotaur" matches 'St. Martins Press' by (68)


In [39]:
check_list_4 = ['Scholastic  Inc.','Scholastic  Inc.', 'Scholastic Inc','Scholastic Books','Scholastic Audio Books',\
 'Scholastic Nonfiction','Scholastic Nonfiction','Scholastic Reference','Scholastic Reference']

In [40]:
checktextSimilarity(check_list_4)

'Scholastic  Inc.' matches 'Scholastic  Inc.' by (100)
'Scholastic  Inc.' matches 'Scholastic Inc' by (97)
'Scholastic  Inc.' matches 'Scholastic Books' by (79)
'Scholastic  Inc.' matches 'Scholastic Audio Books' by (79)
'Scholastic  Inc.' matches 'Scholastic Nonfiction' by (79)
'Scholastic  Inc.' matches 'Scholastic Nonfiction' by (79)
'Scholastic  Inc.' matches 'Scholastic Reference' by (79)
'Scholastic  Inc.' matches 'Scholastic Reference' by (79)


In [41]:
check_list_5 = ['Random House  for Young Readers','Random House  for Young Readers','Random House Books for Young Readers',\
'Random House for Young Readers','Random House Audio','Random House Audio',"Random House Children's Books",\
 "Random House Children's Books",'Random House Large Print','Random House Large Print',\
 'Random House Mondadori','Random House Mondadori','Random House Puzzles & Games','Random House Puzzles & Games',\
 'Random House Trade','Random House Trade','Random House UK Distribution','Random House UK Distribution',\
 'Random House Value Publishing','Random House Value Publishing',\
 'Random House Vintage','Random House (Vintage)','Random House Vintage','Vintage','Vintage/Random House (NY)']

In [42]:
checktextSimilarity(check_list_5)

'Random House  for Young Readers' matches 'Random House  for Young Readers' by (100)
'Random House  for Young Readers' matches 'Random House Books for Young Readers' by (95)
'Random House  for Young Readers' matches 'Random House for Young Readers' by (98)
'Random House  for Young Readers' matches 'Random House Audio' by (86)
'Random House  for Young Readers' matches 'Random House Audio' by (86)
'Random House  for Young Readers' matches "Random House Children's Books" by (58)
'Random House  for Young Readers' matches "Random House Children's Books" by (58)
'Random House  for Young Readers' matches 'Random House Large Print' by (64)
'Random House  for Young Readers' matches 'Random House Large Print' by (64)
'Random House  for Young Readers' matches 'Random House Mondadori' by (68)
'Random House  for Young Readers' matches 'Random House Mondadori' by (68)
'Random House  for Young Readers' matches 'Random House Puzzles & Games' by (64)
'Random House  for Young Readers' matches 'Random Ho